In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
customers = pd.read_csv('Olist/olist_customers_dataset.csv')
orders = pd.read_csv('Olist/olist_orders_dataset.csv')
payments = pd.read_csv('Olist/olist_order_payments_dataset.csv')
reviews = pd.read_csv('Olist/olist_order_reviews_dataset.csv')
items = pd.read_csv('Olist/olist_order_items_dataset.csv')
sellers = pd.read_csv('Olist/olist_sellers_dataset.csv')
products = pd.read_csv('Olist/olist_products_dataset.csv')
translations = pd.read_csv('Olist/product_category_name_translation.csv')

In [3]:
# Variables to Add:
#     Average Review Score
#     Total Sales
#     Total Revenue
#     Seller's Main Product Category
#     Sale Frequency
#     Specialization

In [4]:
# Getting Average Score

# Average review score for all products provided by the seller may 
# determine the quality of that brand.
# ------------------------------------------------------------------------

# 1) First we have to get the review_score from our review dataframe, and 
#    move it to the items dataframe. To do this, it can be passed through
#    the orders dataframe, which is connected through the order_id.
orders = orders.set_index('order_id').join(reviews[['order_id','review_score']].set_index('order_id')).reset_index()
orders.drop_duplicates(inplace=True)
# 2) Much like step one, the review score data is moved from orders to items.
items = items.set_index('order_id').join(orders[['order_id','review_score']].set_index('order_id')).reset_index()
items.drop_duplicates(inplace=True)
# 3) After grouping the reviews by seller_id, it can be aggregated and averaged
#    where it, along with the seller id, are stored into a new dataframe.
average_review_score = pd.DataFrame(items.groupby(by='seller_id')['review_score'].agg('mean')).reset_index()
average_review_score.rename(columns={'review_score':'average_review_score'})
# 4) the data is finally moved to the sellers dataset.
sellers = sellers.set_index('seller_id').join(average_review_score[['seller_id','review_score']].set_index('seller_id')).reset_index()

In [5]:
# Getting Total Orders <total_orders>

# The total amount of orders that a Seller has sold is another standard
# varaible we can create
# ------------------------------------------------------------------------
# 1) group items by seller_id, count them, and save it into total_sales
total_sales = items.groupby(by='seller_id')['review_score'].count().reset_index()
total_sales.rename(columns={'review_score':'total_sales'},inplace=True)
sellers.reset_index()
# 2) Add it back into the sellers dataset
sellers = sellers.reset_index().set_index('seller_id').join(total_sales.set_index('seller_id'))

In [6]:
# Getting Total Revenue: <total_revenue>

# Getting the total revenue that each seller made will be a great variable 
# to use.
# ------------------------------------------------------------------------
total_price = pd.DataFrame(items.groupby(by='seller_id')['price'].sum())
total_price.rename(columns={'price':'total_revenue'},inplace=True)
total_price.reset_index()
sellers = sellers.reset_index().set_index('seller_id').join(total_price.reset_index()[['seller_id','total_revenue']].set_index('seller_id')).reset_index()

In [7]:
# Translations of Products:

# For ease of analysis, all products in portugese will be translated to english.
# ------------------------------------------------------------------------
products = products.set_index('product_category_name').join(translations.set_index('product_category_name')).reset_index()
products.dropna(inplace=True)
products.pop('product_category_name')

0        agro_industria_e_comercio
1        agro_industria_e_comercio
2        agro_industria_e_comercio
3        agro_industria_e_comercio
4        agro_industria_e_comercio
                   ...            
32336        utilidades_domesticas
32337        utilidades_domesticas
32338        utilidades_domesticas
32339        utilidades_domesticas
32340        utilidades_domesticas
Name: product_category_name, Length: 32327, dtype: object

In [8]:
# Getting Seller Main Category: <favorite_product>

# Getting the seller's most popular product category will allow us to 
# measure how specialized that seller is. This may reflect a seller's 
# success or vise versa.
# ------------------------------------------------------------------------
# 1) Move the english product names into the items dataset
items = items.set_index('product_id').join(products[['product_id','product_category_name_english']].set_index('product_id')).reset_index()
items.fillna('unknown',inplace=True)
# 2) A seller's favorite product is calculated by grouping every item sold by their seller and selecting the category
#    that sold the most.
favorite_products = pd.DataFrame(items.groupby(by='seller_id')['product_category_name_english'].agg(lambda x: pd.Series.mode(x)[0]))
favorite_products.rename(columns={'product_category_name_english':'favorite_product'},inplace=True)
sellers = sellers.set_index('seller_id').join(favorite_products.reset_index().set_index('seller_id')).reset_index()

In [9]:
# Getting Frequency: <average_frequency>

# We want to roughly measure the stability of a given seller which may 
# provide as useful tool for seller targeting.
# ------------------------------------------------------------------------
# 1) move time of purchase from orders DS to items DS
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
order_date = orders[['order_id','order_purchase_timestamp']]
items = items.set_index('order_id').join(order_date.set_index('order_id')).reset_index()
items.drop_duplicates(inplace=True)
items.rename(columns={'order_purchase_timestamp':'product_purchase_timestamp'},inplace=True)
# 2) Create a variable that checks whether sellers have multiple sales and move to the items dataframe
multiple_sales = pd.DataFrame(items.groupby(by='seller_id')['product_purchase_timestamp'].count() > 1)
multiple_sales.rename(columns={'product_purchase_timestamp':'multiple_sales'},inplace=True)
items = items.set_index('seller_id').join(multiple_sales.reset_index()[['seller_id','multiple_sales']].set_index('seller_id')).reset_index()
# 3) If seller has multiple sales, then create a new variable that gets their average sale frequency
time_dis = pd.DataFrame(items.loc[items.multiple_sales == True].groupby(by=['seller_id'])['product_purchase_timestamp'].diff())
time_dis = time_dis.abs()
time_dis.product_purchase_timestamp = time_dis.product_purchase_timestamp.dt.days
items['time_since_last'] = time_dis
average_frequency = pd.DataFrame(items.groupby(by=['seller_id'])['time_since_last'].mean())
average_frequency.rename(columns={'time_since_last':'average_frequency'},inplace=True)
average_frequency.reset_index()
average_frequency = average_frequency.reset_index().set_index('seller_id').join(items[['seller_id','multiple_sales']].reset_index().set_index('seller_id')).reset_index()
average_frequency.drop_duplicates(keep='last',subset=['seller_id'],inplace=True)
average_frequency.pop('index')
# 4) Move the new average frequency dataset to the sellers DS
sellers = sellers.set_index('seller_id').join(average_frequency[['seller_id','average_frequency','multiple_sales']].set_index('seller_id')).reset_index()
sellers.pop('index')
sellers.drop_duplicates(subset=['seller_id'],inplace=True)
sellers['multiple_sales'] = sellers['multiple_sales'].astype(int)
sellers["average_frequency"].fillna(value=735, inplace=True)
sellers.reset_index()

,index,seller_id,seller_zip_code_prefix,seller_city,seller_state,review_score,total_sales,total_revenue,favorite_product,average_frequency,multiple_sales
0,0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,3.000000,3,218.70,sports_leisure,112.000000,1
1,1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,4.560976,41,11703.07,luggage_accessories,99.225000,1
2,2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,5.000000,1,158.00,baby,735.000000,0
3,3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,5.000000,1,79.99,sports_leisure,735.000000,0
4,4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,1.000000,1,167.99,electronics,735.000000,0
...,...,...,...,...,...,...,...,...,...,...,...
3090,3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR,5.000000,2,158.00,housewares,9.000000,1
3091,3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC,4.100000,10,889.00,cool_stuff,173.111111,1
3092,3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP,5.000000,7,550.04,sports_leisure,42.333333,1
3093,3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,4.454545,11,297.00,unknown,22.900000,1


In [10]:
# Getting Seller's Specialization Ratio: <main_product_ratio>

# Using the <favorite_product> variable, we can find the ratio between the 
# total count of a seller's favorite product and the total count of 
# products sold by that seller.
# ------------------------------------------------------------------------

# 1) move favorite product from sellers into items dataset and count all the
#    sales for that favorite product for every seller
items = items.set_index('seller_id').join(sellers[['seller_id','favorite_product']].set_index('seller_id')).reset_index()
main_product_sales = pd.DataFrame(items.loc[items.product_category_name_english==items.favorite_product].groupby(by='seller_id')['favorite_product'].count())
main_product_sales.rename(columns={'favorite_product':'main_product_sales'},inplace=True)
# 2) get a total count of all products sold for every seller
all_sales = pd.DataFrame(items.groupby(by='seller_id')['favorite_product'].count())
all_sales.rename(columns={'favorite_product':'all_sales'},inplace=True)
main_product_sales.reset_index()
all_sales.reset_index()
# 3) get the ratio between count of favorite products and total products sold
#    for every seller, and then move it to the sellers dataset.
product_ratio = all_sales.reset_index().set_index('seller_id').join(main_product_sales.reset_index().set_index('seller_id')).reset_index()
product_ratio['main_product_ratio'] = (product_ratio.main_product_sales / product_ratio.all_sales) * 100
sellers = sellers.set_index('seller_id').join(product_ratio[['seller_id','main_product_ratio']].set_index('seller_id')).reset_index()

In [11]:
# Dropped Off

# We want to a varaible that measures whether a seller has recently been 
# selling at their expected frequency. This may be a rouch metric for 
# seller stability.
# ------------------------------------------------------------------------

# 1) Sort items by seller_id and then by product_purchase_timestamp. Then
#    keep only the latest seller timestamps.
items_temp = items.sort_values(by=['seller_id','product_purchase_timestamp'])
items_temp.drop_duplicates(keep='last',subset=['seller_id'],inplace=True)
items_temp = items_temp.reset_index()
# 2) Get the latest date and subtract the latest product_purchase_timestamp
#    by this date, convert it into days, and save this into the sellers dataframe
latest_date = dt.datetime.strptime('2018-09-09 23:59:59', '%Y-%m-%d %H:%M:%S')
items_temp['days_since_last'] = (latest_date - items_temp.product_purchase_timestamp).dt.days
sellers = sellers.set_index('seller_id').join(items_temp[['seller_id','days_since_last']].set_index('seller_id')).reset_index()
# 3) Finally, return the boolean operation to whether this value is larger than 
#    the average_frequency.
sellers['dropped_off'] = sellers['average_frequency'] < sellers['days_since_last']

In [12]:
# Customer Collaborative Filtering Preparation:

# In order to use Collaborative Filtering, A dataset containing customer id,
# review_score, and product_name must be created. This is most easily done
# by moving customer ID into the items dataset. Due to past cleaning and
# joining, both seller_id and review_score are already held within the 
# items dataframe.
# ------------------------------------------------------------------------

# 1) move customer_id into orders
orders = orders.set_index('customer_id').join(customers[['customer_id','customer_unique_id']].set_index('customer_id'))
orders = orders.reset_index()
# 2) move customer_id into items
items = items.set_index('order_id').join(orders[['order_id','customer_unique_id']].set_index('order_id'))
customer_items = items[['customer_unique_id','product_category_name_english','review_score']]
customer_items = customer_items.reset_index()
customer_items.pop('order_id')
customer_items.rename(columns={'product_category_name_english':'product_name'},inplace=True)

# Save the full Sellers dataset to a clean csv file
cus_items.to_csv('CleanedDatasets/cus_items.csv')

In [13]:
# Save the full Sellers dataset to a clean csv file
sellers.to_csv('CleanedDatasets/sellers_c.csv')


In [14]:
# Additional aggreagated statistics:
# ------------------------------------------------------------------------

In [15]:
# Average total sales
sellers.total_sales.mean()

36.48529886914378

In [16]:
# Proportion with only 1 sale
sellers.loc[sellers.total_sales==1]['total_sales'].count() / sellers['total_sales'].count() * 100

16.31663974151858

In [18]:
cus_items

product_name,agro_industry_and_commerce,air_conditioning,art,arts_and_craftmanship,audio,auto,baby,bed_bath_table,books_general_interest,books_imported,...,signaling_and_security,small_appliances,small_appliances_home_oven_and_coffee,sports_leisure,stationery,tablets_printing_image,telephony,toys,unknown,watches_gifts
customer_unique_id,,,,,,,,,,,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000b849f77a49e4a4ce2b2a4ca5be3f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000f46a3911fa3c0805444483337064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
0000f6ccb0745a6a4b88665a16c9f078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
0004aac84e0df4da2b147fca70cf8255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fffea47cd6d3cc0a88bd621562a9d061,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffff371b4d645b6ecea244b27531430a,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
